In [1]:
print('hello')

hello


In [2]:
##### IMPORTS #####

import numpy as np

import pandas as pd

from functools import partial

import math

import operator

import itertools

##### IMPORTS #####

In [3]:
# Read the dataset
df_train = pd.read_csv("Notebook_Results/TRAIN-ENG-GER-FRE-N3-bin-informative.csv", index_col='Unnamed: 0')
df_train = df_train.dropna()
df_train = df_train.reset_index(drop=True)
# print("df_train")
# print(df_train); print(df_train.dtypes)
df_test = pd.read_csv("Notebook_Results/TEST-ENG-GER-FRE-N3-bin-informative.csv", index_col='Unnamed: 0')
df_test = df_test.dropna()
df_test = df_test.reset_index(drop=True)
# print("df_test")
# print(df_test); print(df_test.dtypes)

In [4]:
feature_list = list(set(df_train)-{'word', 'word_length', 'label'})
print(feature_list)
print(len(feature_list))

['word_NTE_gram_bin', 'word_C_gram_bin', 'word_K_gram_bin', 'word_ER_gram_bin', 'word_É_gram_bin', 'word_G_gram_bin', 'word_E_gram_bin', 'word_TE_gram_bin', 'word_Ä_gram_bin', 'word_TER_gram_bin', 'word_N_gram_bin', 'word_S_gram_bin', 'word_ERN_gram_bin', 'word_H_gram_bin', 'word_EI_gram_bin', 'word_NT_gram_bin', 'word_TEN_gram_bin', 'word_EN_gram_bin', 'word_Y_gram_bin', 'word_STE_gram_bin']
20


In [5]:
label_grouped = df_train.groupby('label').count()
label_values = df_train['label'].unique()
label_sum = sum(label_grouped['word'])
label_probs = dict.fromkeys(label_values, 0)
for label_val in label_values:
    label_probs[label_val] = label_grouped['word'][label_val]/label_sum
print(label_probs)

{2: 0.3742382412437501, 1: 0.47129330415643567, 3: 0.15446845459981426}


In [6]:
feature_probs = np.zeros((20, 2, 3))
for i in range(len(feature_list)):
    feat = feature_list[i]
    df_zip = df_train[[feat, 'label', 'word']]
    grouped = df_zip.groupby([feat, 'label']).count()
    levels = []
    for j in range(2):
        levels.append(grouped.index.levels[j].values)
    new_index = pd.MultiIndex.from_product(levels, names=grouped.index.names)
    grouped = grouped.reindex(new_index, fill_value=0)
    for k in range(2):
        for l in range(1,4):
            pxny = grouped['word'][k][l]/label_sum
            pxIy = pxny/label_probs[l]
            feature_probs[i][k][l-1] = pxIy
    print(grouped)
print(feature_probs)

                           word
word_NTE_gram_bin label        
0                 1      291747
                  2      227191
                  3       94331
1                 1        4606
                  2        8133
                  3        2800
                         word
word_C_gram_bin label        
0               1      193301
                2      153492
                3       68411
1               1      103052
                2       81832
                3       28720
                         word
word_K_gram_bin label        
0               1      276007
                2      165549
                3       96376
1               1       20346
                2       69775
                3         755
                          word
word_ER_gram_bin label        
0                1      245313
                 2      137421
                 3       81232
1                1       51040
                 2       97903
                 3       15899
                

In [9]:
## Testing on sample rows ##
samp_row = df_test.sample(1)
# samp_lab_prob = np.ones((1,3), dtype=int)
# samp_lab_prob = [label_probs[1], label_probs[2], label_probs[3]]
samp_lab_prob = {1:label_probs[1], 2:label_probs[2], 3:label_probs[3]}
# print(samp_lab_prob)
# print(samp_lab_prob[2])
# print(feature_probs)
# print(feature_probs[19][1][2])
for lab in range(1,4):
    for i in range(len(feature_list)):
        feat = feature_list[i]
        temp1 = feature_probs[i]
#         print(temp1)
        temp2 = temp1[samp_row[feat]][0]
#         print(temp2)
        temp3 = temp2[lab-1]
#         print(temp3)
        pxIy = temp3
#         samp_lab_prob[lab-1] = samp_lab_prob[lab-1] * pxIy
        samp_lab_prob[lab] = samp_lab_prob[lab] * pxIy
print(samp_row['label'])
print(samp_lab_prob)
print(max(samp_lab_prob, key=samp_lab_prob.get))

104758    1
Name: label, dtype: int64
{1: 0.0029092472457682315, 2: 0.0019500819019741416, 3: 0.0009495872719404489}
1


In [10]:
df_test['p_label'] = 0
for ind in range(len(df_test.index)):
    samp_lab_prob = {1:label_probs[1], 2:label_probs[2], 3:label_probs[3]}
    row = df_test.loc[[ind]]
    for lab in range(1,4):
        for i in range(len(feature_list)):
            feat = feature_list[i]
            pxIy = feature_probs[i][row[feat]][0][lab-1]
            samp_lab_prob[lab] = samp_lab_prob[lab] * pxIy
#     row['p_label'] = max(samp_lab_prob, key=samp_lab_prob.get)
    pred = max(samp_lab_prob, key=samp_lab_prob.get)
    df_test.set_value(ind, 'p_label', pred)
#     print(ind)
#     print(df_test.loc[[ind]]['p_label'])
#     print(df_test.loc[[ind]]['label'])

C:\Users\MONSTER\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':


In [11]:
mistak = 0
total = len(df_test.index)
tp1 = 0
tp2 = 0
tp3 = 0
fn1 = 0
fn2 = 0
fn3 = 0
fp1 = 0
fp2 = 0
fp3 = 0
total1 = 0
total2 = 0
total3 = 0
for ind in range(len(df_test.index)):
    row = df_test.loc[[ind]]
    l = row['label'][ind]
    p = row['p_label'][ind]
    if l == 1:
        total1 = total1 + 1
        if p == 2:
            fn1 = fn1 + 1
            fp2 = fp2 + 1
        elif p == 3:
            fn1 = fn1 + 1
            fp3 = fp3 + 1
        elif p == 1:
            tp1 = tp1 + 1
    elif l == 2:
        total2 = total2 + 1
        if p == 1:
            fn2 = fn2 + 1
            fp1 = fp1 + 1
        elif p == 3:
            fn2 = fn2 + 1
            fp3 = fp3 + 1
        elif p == 2:
            tp2 = tp2 + 1
    else:
        total3 = total3 + 1
        if p == 1:
            fn3 = fn3 + 1
            fp1 = fp1 + 1
        elif p == 2:
            fn3 = fn3 + 1
            fp2 = fp2 + 1
        elif p == 3:
            tp3 = tp3 + 1
        
    if l != p:
        mistak = mistak + 1
#     if ind % 25000 == 0:
#         print(ind)
print("# of wrong predictions %s" % (mistak))
print("# of samples %s" % (total))
print("Accuracy %s" % (mistak/total))
print('English tp fn fp tn set-size')
print(tp1)
print(fn1)
print(fp1)
print(total - tp1 - fn1 - fp1)
print(total1)
print('German tp fn fp tn set-size')
print(tp2)
print(fn2)
print(fp2)
print(total - tp2 - fn2 - fp2)
print(total2)
print('French tp fn fp tn set-size')
print(tp3)
print(fn3)
print(fp3)
print(total - tp3 - fn3 - fp3)
print(total3)
print('English recall precision')
r1 = tp1 / (tp1 + fn1)
p1 = tp1 / (tp1 + fp1)
print(tp1 / (tp1 + fn1))
print(tp1 / (tp1 + fp1))
print('German recall precision')
r2 = tp2 / (tp2 + fn2)
p2 = tp2 / (tp2 + fp2)
print(tp2 / (tp2 + fn2))
print(tp2 / (tp2 + fp2))
print('French recall precision')
r3 = tp3 / (tp3 + fn3)
p3 = tp3 / (tp3 + fp3)
print(tp3 / (tp3 + fn3))
print(tp3 / (tp3 + fp3))

# of wrong predictions 40015
# of samples 156257
Accuracy 0.2560845274131719
English tp fn fp tn set-size
62274
11469
25938
56576
73743
German tp fn fp tn set-size
45248
13221
12553
85235
58469
French tp fn fp tn set-size
8720
15325
1524
130688
24045
English recall precision
0.8444733737439486
0.7059583730104748
German recall precision
0.7738801758196652
0.7828238265773949
French recall precision
0.36265335828654605
0.8512299882858259


In [12]:
avgr = (r1 + r2 + r3) / 3
avgp = (p1 + p2 + p3) / 3
# print(avgr)
# print(avgp)
f1 = 2 * (avgp * avgr) / (avgp + avgr)
print("F1 score: %s" % (f1))

F1 score: 0.7151986149469226


In [13]:
# Naive bayes p(y|x) ~ p(x1|y)...p(xp|y)p(y)